# Configuración Inicial
Métodos un utils en común en los notebooks

In [1]:
# Imports 
from pyspark.sql.types import IntegerType, StringType, DateType, LongType
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, length


In [2]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):        
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df
    
    def save_db(self, df, tabla):
        df.write.jdbc(
            url=self.url,
            table=tabla,
            mode='append',
            properties=self.properties
        )
        
def create_spark_session(path_jar_driver):    
    conf = SparkConf().set('spark.driver.extraClassPath', path_jar_driver)
    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    return sql_context.sparkSession    

def get_dataframe_from_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

In [4]:
db_user = 'Estudiante_65_202415'
db_psswd = 'Estudiante_202010409'

connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver"
}

source_db_string_connection = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional_ETL'
destination_db_string_connection = f'jdbc:mysql://157.253.236.120:8080/{db_user}'

# Driver de conexion
# LINUX
path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'
# WINDOWS
#path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [5]:
spark = create_spark_session(path_jar_driver)

24/11/18 20:45:39 WARN Utils: Your hostname, willp resolves to a loopback address: 127.0.1.1; using 192.168.0.6 instead (on interface enp8s0)
24/11/18 20:45:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 20:45:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/18 20:45:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/willp/anaconda3/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
conn_orig = MySQLConnector(spark=spark, connection_properties=connection_properties, url=source_db_string_connection)
conn_dest = MySQLConnector(spark=spark, connection_properties=connection_properties, url=destination_db_string_connection)

# Consultar Dimensiones

In [12]:
# Areas de Servicio
sql_dim_areas_servicio = '''(SELECT IdAreaDeServicio_DWH, IdAreaDeServicio_T FROM Rs_AreasDeServicio ORDER BY IdAreaDeServicio_DWH, IdAreaDeServicio_T) AS AreasDeServicio'''
dim_areas_servicio = conn_dest.get_dataframe(sql_dim_areas_servicio)

In [11]:
# Mini condiciones
sql_dim_condiciones = '''(SELECT IdCondicionesBeneficios_DWH, IdTipoBeneficio_T, Annio FROM Rs_MiniCondicionesTipoBeneficio ORDER BY IdCondicionesBeneficios_DWH, IdTipoBeneficio_T, Annio) AS MiniCondicionesTipoBeneficio'''
dim_mini_condiciones = conn_dest.get_dataframe(sql_dim_condiciones)

In [14]:
# Proveedor
sql_dim_proveedores = '''(SELECT IdProveedor_DWH, IdProveedor_T FROM Rs_Proveedor ORDER BY IdProveedor_DWH, IdProveedor_T) AS Proveedor'''
dim_proveedores = conn_dest.get_dataframe(sql_dim_proveedores)

In [15]:
# Niveles de Servicio
sql_dim_nivel_servicio = '''(SELECT IdNivelDeServicio_DWH, IdNivelDeServicio_T FROM Rs_NivelesDeServicio ORDER BY IdNivelDeServicio_DWH, IdNivelDeServicio_T) AS NivelesDeServicio'''
dim_nivel_servicio = conn_dest.get_dataframe(sql_dim_nivel_servicio)

In [16]:
# Tipos Beneficio
sql_dim_tipos_beneficio = '''(SELECT IdTipoBeneficio_DWH, IdTipoBeneficio_T FROM Rs_TiposBeneficio ORDER BY IdTipoBeneficio_DWH, IdTipoBeneficio_T) AS TiposBeneficio'''
dim_tipos_beneficio = conn_dest.get_dataframe(sql_dim_tipos_beneficio)

In [ ]:
# Fecha
sql_dim_fecha = '''(SELECT IdTipoBeneficio_DWH, IdTipoBeneficio_T FROM Rs_TiposBeneficio ORDER BY IdTipoBeneficio_DWH, IdTipoBeneficio_T) AS TiposBeneficio'''
dim_tipos_beneficio = conn_dest.get_dataframe(sql_dim_tipos_beneficio)

# Hecho Planes tipo Beneficio